This program creates box plots for the output from the TIME web application's Workflow 5b, which caluclates uses their custom Dynamic Time Warping (DTW) algorithm to calculate the pairwise DTW distance between bacterial taxa. Dynamic Time Warping is a measure of similarity in longitudinal data, and the TIME version of the algorithm ranges from 0 to 1. 0 is the most similar, and 1 is the most different. For more on the algorithm, see the relevant research paper [here](https://www.frontiersin.org/articles/10.3389/fmicb.2018.00036/full).
<br>

For the CF data, I used the following settings: a taxonomic level of 'Genus,' the default constraint of 2 and a filter for rare taxa of 0.1 (because the input data was already filtered). The input file was not rarefied or normalised in the intitial upload. In the programs Create_TDTW_all_filtered and Create_TDTW_all_example I created csv files for Workflow 5b output data. Now it's time to plot them, with a specific focus on 3 bacteria of particular interest to our CF research. 

This program is written for use with the CF data, but to make it possible for users to run it I have included in this repository an altered version of the output from Create_TDTW_all_filtered, called TDTW_all_filtered2.csv, using randomly generated values. If you are interested in seeing how I created TDTW_all_filtered2.csv, please email me at vtalbot@lesley.edu or vrtalbot@yahoo.com and I will be happy to send you the program.  

Users also have the option of doing everything themselves in making the box plots, from running the TIME workflow on their example [repeated antibiotic perturbation](https://web.rniapps.net/time/index.php) data to creating the merged file in Create_TDTW_all_example and making the box plots in DTW_All_boxplots_example. Create_TDTW_all example also produces input for DTW_boxplots_by_status, which does not have a counterpart for the CF data. 

This code may easily be adapted to create box plots for other types of data. It plots a selection of columns from the table rather than grouping by a factor variable (See DTW_boxplots_by_status for box plots which *are* grouped by a factor variable). It also extracts names of bacteria from much longer strings. See the program DTW_All_boxplots_example for the same code using the example file generated in Create_TDTW_all_example, which you can run for yourself.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
#make a list of the OTU's which are of special interest to us, which we wish to plot
bacteria=['Pseudomonas','Staphylococcus','Streptococcus']

## Section 1
The first few cells modify the main dataset created in Create_TDTW_all_filtered to focus on only 3 taxa, creating a separate file for each of the three which shows the distance from the other bacteria to that one bacteria. You can, of course, easily edit this to focus on any number of selected taxa. 

<br>

You can run these cells every time you use this program, or save the filtered files after creating them for the bacteria you wish to plot, in which case you can skip down to Section 2. There are alternate versions of the code for both sets of plots that reads them in to plot them if they are saved to your computer.

In [ ]:
#read in the file, created in the program Create_TDTW_all_filtered, which contains all the output from Workflow 5b
#here I am using a randomly generated version of TDTW_all_filtered.csv called TDTW_all_filtered2.csv instead
df = pd.read_csv("Data/TDTW_all_filtered2.csv")

In [ ]:
#rename the first column, which has no name, to 'Sample' 
df.rename(columns={'Unnamed: 0': 'Sample'}, inplace=True)


In [ ]:
#extract the participant's clinical status from the sample identifier, and change to title case
#I used lower case when I created the sample identifiers because they were easier to read that way
df['Status']=df['Sample'].str.split('_').str[2].str.title()


In [ ]:
#extract the participant's ID number from the identifier
df['ID']=df['Sample'].str.split('_').str[1]
#review the changes to confirm that they worked
df.head()

What the new columns should look like:
<img src='https://imgur.com/KU0nqUf.png' style='height:175px'>

In [ ]:
#create a separate data frame for just those entries with status 'All'
df2=df[df['Status'] == 'All']
#view the head of the new table if desired
df2.head()

Note that in our CF input data for TIME, taxonomic names are preceded by double underscores at all levels.
<br>

Example: D_0__Bacteria;D_1__Actinobacteria;D_2__Actinobacteria;D_3__Actinomycetales;D_4__Actinomycetaceae;D_5__Actinomyces
<br>

This generates output for the genus level with Actinomyces preceded by double underscores in every workflow.
We left the underscores in as separators when we merged the output files from TIME in Create_TDTW_all_filtered. The example data lacked these underscores, so we chose to add a single underscore as a separator between bacteria names in Create_TDTW_all_example. This means that, if you are editing this program to run it on your own data, the code below may need to be modified to reflect any formatting differences. 

In [ ]:
#make a list of columns (pairs of OTU's) for only the 3 OTU's of interest, excluding the ones that pair them with themselves 
clist=[]
for i in range(0,3):
    #if you are editing this code to use on your own data, here you may need a minor edit to relect formatting differences
    #viewing the head of df or df2 above should tell you if and how you need to change the str.format() function
    j=[col for col in df.columns if col.startswith('__{}'.format(bacteria[i])) 
       and col!= '__{}__{}'.format(bacteria[i],bacteria[i])]
    clist.append(j)                                              
#view the list to check it, if desired
#print(clist)

## Section 2

I included the code in all of my different plotting programs for GP Microbiome output the creation of keys for bacteria names. The keys pair OTU ID numbers with names of bacteria. The file 'OTU_key_named_selection.csv' contains only the bacteria of particular interest to the CF study. 'OTU_key_named_selection.xlsx' is that file, with information added about the NCBI taxonomic ID and how we want the boxplot to look, saved as an Excel workbook. 

For quick basic plots without including the NCBI taxonomic ID's or specifying the order, skip to the last two cells. To specify the order without showing the taxonomic ID's, simply comment out the line in the loop which renames the columns.

## Plots with Taxonomic ID's

In [ ]:
#read in the file and take a look at the columns
#remember to adjust the file path for your computer
key=pd.read_excel("Data/OTU_key_named_selection.xlsx")
key.head()


In [ ]:
#create and view a dictionary for the taxonomic ID's
taxid_key=dict(zip(key['Name'], key['NCBI:txid']))
#view dictionary, if desired
taxid_key

In [ ]:
#sort by plotting order
taxa=key[key['in_boxplot']=='Y'].sort_values('order for plotting')
#view, if desired
taxa.head()

Loop that creates the files:

In [ ]:
for i in range(0,3):
    df3=df2.filter(clist[i], axis=1)
    #since each data frame is for the distance from one bacteria, change the column names to be just the second bacteria
    #this code may need adjustment depending on how many underscores are in your bacteria names
    df3.columns=[w.replace('__{}__'.format(bacteria[i]),'') for w in df3.columns]
    #save file if desired, adjusting the file path for your computer
    #df3.to_csv("Data/TDTW_filtered_{}.csv".format(bacteria[i]), index=False)
    #to keep those with at least 4 non-NaN's:
    #df3.dropna(thresh=4, axis=1, inplace=True)
    df3.dropna(axis=1, inplace=True)
    order=list(taxa['Name'][taxa['Name']!=bacteria[i]])
    df3=df3.reindex(columns=order)
    df3.columns=[col+' NCBI:taxid {}'.format(str(taxid_key[col])) for col in df3.columns]
    plt.figure(figsize=(12,14))
    ax=sns.boxplot(x="variable", y="value", data=pd.melt(df3))
    plt.setp(ax.get_xticklabels(), rotation=90, size=18, style='italic')
    plt.setp(ax.get_yticklabels(), size=16)
    plt.title('DTW Distance from {}'.format(bacteria[i]), size=24)
    plt.xlabel('')
    plt.ylabel('Distance',size=20)
    plt.tight_layout()
    #save plot if desired - you might want to put it in its own folder
    #plt.savefig("Plots/{}_taxid.png".format(bacteria[i]), format = "png")
    plt.show()

Loop that reads in the files:

In [ ]:
#loop for if the files have already been created
for i in range(0,3):
    #remember to adjust the file path when reading in the file
    df = pd.read_csv("Data/TDTW_filtered_{}.csv".format(bacteria[i]))
    df.dropna(axis=1, inplace=True)
    order=list(taxa['Name'][taxa['Name']!=bacteria[i]])
    df=df.reindex(columns=order)
    df.columns=[col+' NCBI:taxid {}'.format(str(taxid_key[col])) for col in df.columns]
    plt.figure(figsize=(12,14))
    ax=sns.boxplot(x="variable", y="value", data=pd.melt(df))
    plt.setp(ax.get_xticklabels(), rotation=90, size=18, style='italic')
    plt.setp(ax.get_yticklabels(), size=16)
    plt.title('DTW Distance from {}'.format(bacteria[i]), size=24)
    plt.xlabel('')
    plt.ylabel('Distance',size=20)
    plt.tight_layout()
    #save plot if desired - you might want to put it in its own folder
    #plt.savefig("Plots/{}_taxid.png".format(bacteria[i]), format = "png")
    plt.show() 

Here is a mini preview of what the three image files from the plots with taxonomic ID's will look like.
<table><tr>
<td> <img src='https://imgur.com/rT8s8AO.png' style='height:400px'></td>
<td><img src='https://imgur.com/X1CDGRN.png' style='height:400px'></td>
<td><img src='https://imgur.com/u0WvG6n.png' style='height:400px'></td>
</tr></table>

## Quick basic plots
Please note that these plots do not specify the order of the bacteria. To specify the order, go back to the beginning of Section 2.

Loop that creates the files:

In [ ]:
for i in range(0,3):
    df3=df2.filter(clist[i], axis=1)
    #since each data frame is for the distance from one bacteria, change the column names to be just the second bacteria
    #this code may need adjustment depending on how many underscores are in your bacteria names    
    df3.columns=[w.replace('__{}__'.format(bacteria[i]),'') for w in df3.columns]
    #save file if desired, adjusting the file path for your computer
    #df3.to_csv("Data/TDTW_filtered_{}.csv".format(bacteria[i]), index=False)
    #to keep those with at least 4 non-NaN's:
    #df3.dropna(thresh=4, axis=1, inplace=True)
    df3.dropna(axis=1, inplace=True)
    plt.figure(figsize=(12,10))
    ax=sns.boxplot(x="variable", y="value", data=pd.melt(df3))
    plt.setp(ax.get_xticklabels(), rotation=90, size=18, style='italic')
    plt.setp(ax.get_yticklabels(), size=16)
    plt.title('DTW Distance from {}'.format(bacteria[i]), size=24)
    plt.xlabel('')
    plt.ylabel('Distance',size=20)
    plt.tight_layout()
    #save plot if desired - you might want to put it in its own folder
    #plt.savefig("Plots/{}_basic.png".format(bacteria[i]), format = "png")
    plt.show() 

Loop that reads in the files:

In [ ]:
#loop for if the files have already been created
for i in range(0,3):
    #remember to adjust the file path when reading in the file
    df = pd.read_csv("Data/TDTW_filtered_{}.csv".format(bacteria[i]))
    df.dropna(axis=1, inplace=True)
    plt.figure(figsize=(12,10))
    ax=sns.boxplot(x="variable", y="value", data=pd.melt(df))
    plt.setp(ax.get_xticklabels(), rotation=90, size=18, style='italic')
    plt.setp(ax.get_yticklabels(), size=16)
    plt.title('DTW Distance from {}'.format(bacteria[i]), size=24)
    plt.xlabel('')
    plt.ylabel('Distance',size=20)
    plt.tight_layout()
    #save plot if desired - you might want to put it iin its own folder
    #plt.savefig("Plots/{}_basic.png".format(bacteria[i]), format = "png")
    plt.show() 

Here is a mini preview of what the three image files from the basic plots will look like.
<table><tr>
<td> <img src='https://imgur.com/6Co1QBL.png' style='height:310px'></td>
<td><img src='https://imgur.com/WCiddYR.png' style='height:310px'></td>
<td><img src='https://imgur.com/7CMgwrf.png' style='height:310px'></td>
</tr></table>